In [1]:
# Install default libraries
import pathlib
import sys

# Import installed modules
import pandas as pd
import numpy as  np

# Import the Python script from the auxiliary folder
sys.path.insert(1, "../../auxiliary")
import data_fetch

# Set a local download path and the URL to the 67P shape model data set
dl_path = "../../kernels/dsk/"

# Set dictionary with 2 different resolutions
comet_models = {"low": "ROS_CG_M001_OSPCLPS_N_V1.OBJ",
                "high": "ROS_CG_M004_OSPGDLR_N_V1.OBJ"}

# Which model?
model_type = "high"

# Shape model DL
dl_url = f"https://naif.jpl.nasa.gov/pub/naif/ROSETTA/kernels/dsk/{comet_models[model_type]}"

# If file not present: download it!

if not pathlib.Path(f"../../kernels/dsk/{comet_models[model_type]}").is_file():
    
    # Download the shape model, create (if needed) the download path and store the data set
    data_fetch.download_file(dl_path, dl_url)

In [2]:
# Load the shape model. The first column lists whether the row is a vertex or face. The second,
# third and fourth column list the coordiantes (vertex) and vertex indices (faces)
comet_67p_shape_obj = pd.read_csv(f"../../kernels/dsk/{comet_models[model_type]}", \
                                  delim_whitespace=True, \
                                  names=["TYPE", "X1", "X2", "X3"])

# Assign the vertices and faces
vertices = comet_67p_shape_obj.loc[comet_67p_shape_obj["TYPE"] == "v"][["X1", "X2", "X3"]].values \
               .tolist()
faces = comet_67p_shape_obj.loc[comet_67p_shape_obj["TYPE"] == "f"][["X1", "X2", "X3"]].values

# The index in the faces sub set starts at 1. For Python, it needs to start at 0.
faces = faces - 1

# Convert the indices to integer
faces = faces.astype(int)

# Convert the numpy array to a Python list
faces = faces.tolist()

In [3]:
# Doing some SPICE magic here!
import spiceypy

# Load the time kernel
spiceypy.furnsh("../../kernels/lsk/naif0012.tls")

# Load the CG related FIXED reference frame
spiceypy.furnsh("../../kernels/fk/ROS_CHURYUMOV_V01.TF")

# Load in the "rebound" kernel
spiceypy.furnsh("../../kernels/spk/LORB_C_G_FIXED_RBD_7_V2_0.BSP")

# Get the kernel, check the cardinality and determine the startin and end times
rebound_kernel = spiceypy.spkcov(spk="../../kernels/spk/LORB_C_G_FIXED_RBD_7_V2_0.BSP",
                                 idcode=-226800)
rebound_kernel_card = spiceypy.wncard(rebound_kernel)
kernel_start_et, kernel_end_et = spiceypy.wnfetd(rebound_kernel,
                                                 rebound_kernel_card-1)

# List that will store the Philae trajectory
philae_pos_vec = []

# Iterate trough the time steps where Philae jumped around
for et_step in np.arange(kernel_start_et+100, kernel_end_et-100, 10):
    
    # Compute the position of Philae w.r.t. the center of 67P
    philae_pos_vec.append(spiceypy.spkgps(targ=-226800,
                                          et=et_step,
                                          ref="67P/C-G_CK",
                                          obs=1000012)[0])
    
# Convert to numpy array
philae_pos_vec = np.array(philae_pos_vec)

In [4]:
# Let's take a look
print(philae_pos_vec)

[[ 2.14663947 -0.94805434  0.47659463]
 [ 2.14890232 -0.94692065  0.47452082]
 [ 2.15115527 -0.94578482  0.47244124]
 ...
 [ 2.44930453 -0.07388973 -0.3452023 ]
 [ 2.44911728 -0.07335983 -0.34545813]
 [ 2.44891806 -0.07282971 -0.3457068 ]]


In [5]:
# Now we want to plot the comet (as last time), but with Philae's trajectory!
import visvis as vv
from visvis import Point, Pointset

# Convert the philae position vector to a visvis Pointset
pp = Pointset(philae_pos_vec)

In [9]:
# Create visvis application
app = vv.use()

# Create the 3 D shape model as a mesh. verticesPerFace equals 3 since triangles define the
# mesh"s surface in this case
shape_obj = vv.mesh(vertices=vertices, faces=faces, verticesPerFace=3)
shape_obj.specular = 0.0
shape_obj.diffuse = 0.9

# Get axes objects
axes = vv.gca()

# Set a black background
axes.bgcolor = "black"

# Deactivate the grid and make the x, y, z axes invisible
axes.axis.showGrid = False
axes.axis.visible = False

# Set some camera settings
# Please note: if you want to "fly" arond the comet with w, a, s, d (translation) and i, j, k, l
# (tilt) replace "3d" with "fly"
axes.camera = "3d"

# Field of view in degrees
axes.camera.fov = 60

# Set default azmiuth and elevation angle in degrees
axes.camera.azimuth = 120
axes.camera.elevation = 25

# Turn off the main light
axes.light0.Off()

# Create a fixed light source
light_obj = axes.lights[1]
light_obj.On()
light_obj.position = (5.0, 5.0, 5.0, 0.0)

# The stuff above was basically from last time. Now, add Philae's trajectory
vv.plot(pp, ls='-', lc="w", lw=5)

# ... and run the application!
app.Run()

In [7]:
# Tasks:
#
# 1. Plot also Philae descent phase
# 2. Create an animation of the descent phase + the Sun's rays